In [1]:
import json
import numpy as np
import os
import cv2
from xml.dom import minidom
from xml.etree.ElementTree import Element, SubElement, ElementTree, dump
import shutil
import xml.etree.ElementTree as ET

In [2]:
# Timestamp format
def __timestamp_format(raw_timestamp):
    raw_decimal_place_len = len(raw_timestamp.split('.')[-1])
    if(raw_decimal_place_len < 9):
        place_diff = 9 - raw_decimal_place_len
        zero_str = ''
        for _ in range(place_diff):
            zero_str = zero_str + '0'
        formatted_timestamp = raw_timestamp.split('.')[0] + '.' + zero_str + raw_timestamp.split('.')[1]
        return float(formatted_timestamp)
    else:
        return float(raw_timestamp)
    
# Load an annotatiom from the radar frame number
def get_annotation_from_id(annotation_id):
    raw_annotations = []
    for object in annotations_json:
        if (object['bboxes'][annotation_id]):
            obj = {}
            obj['id'] = object['id']
            obj['class_name'] = object['class_name']
            obj['bbox'] = object['bboxes'][annotation_id]
            raw_annotations.append(obj)
    return raw_annotations

def __get_correct_lidar_id_from_raw_ind(id):
    return id-1

def RX(LidarToCamR):
    thetaX = np.deg2rad(LidarToCamR[0])
    Rx = np.array([[1, 0, 0], [0, np.cos(thetaX), -np.sin(thetaX)], [0, np.sin(thetaX), np.cos(thetaX)]]).astype(np.float)
    return Rx

def RY(LidarToCamR):
    thetaY = np.deg2rad(LidarToCamR[1])
    Ry = np.array([[np.cos(thetaY), 0, np.sin(thetaY)],[0, 1, 0],[-np.sin(thetaY), 0, np.cos(thetaY)]])
    return Ry

def RZ(LidarToCamR):
    thetaZ = np.deg2rad(LidarToCamR[2])
    Rz = np.array([[np.cos(thetaZ), -np.sin(thetaZ), 0],[np.sin(thetaZ), np.cos(thetaZ), 0],[0, 0, 1]]).astype(np.float)
    return Rz

def transform(LidarToCamR, LidarToCamT):
    Rx = RX(LidarToCamR)
    Ry = RY(LidarToCamR)
    Rz = RZ(LidarToCamR)
    
    R = np.array([[1, 0, 0], [0, 0, 1], [0, -1, 0]]).astype(np.float)
    R = np.matmul(R, np.matmul(Rx, np.matmul(Ry, Rz)))
    
    LidarToCam = np.array([[R[0, 0], R[0, 1], R[0, 2], 0.0],
                               [R[1, 0], R[1, 1], R[1, 2], 0.0],
                               [R[2, 0], R[2, 1], R[2, 2], 0.0],
                               [LidarToCamT[0], LidarToCamT[1], LidarToCamT[2], 1.0]]).T
    return LidarToCam

def transform_annotations(annotations, M):
    new_annotations = []
    for object in annotations:
        new_object = object
        xx = object['bbox']['position'][0]
        yy = object['bbox']['position'][1]
        sensors_height = -1.8
        zz = sensors_height
        pos = np.array([xx, yy, zz, 1])
        new_pos = np.matmul(M, pos)
        new_pos = new_pos/new_pos[3]
        new_object['bbox']['position'][0] = pos[0]
        new_object['bbox']['position'][1] = pos[1]
        new_annotations.append(new_object)
    return new_annotations

def get_lidar_annotations(id_radar, interp=False, t_c=None, t_r1=None, t_r2=None):
    lidar_annotation_id = __get_correct_lidar_id_from_raw_ind(id_radar)
    lidar_annotations = get_annotation_from_id(lidar_annotation_id)
    if interp and len(get_annotation_from_id(lidar_annotation_id+1)) > 0:
        lidar_annotations_next = get_annotation_from_id(lidar_annotation_id+1)
        
        for ii in range(len(lidar_annotations)):
            try:
                p1x = lidar_annotations[ii]['bbox']['position'][0]
                p1y = lidar_annotations[ii]['bbox']['position'][1]
                p2x = lidar_annotations_next[ii]['bbox']['position'][0]
                p2y = lidar_annotations_next[ii]['bbox']['position'][1]
                lidar_annotations[ii]['bbox']['position'][0] = self.__linear_interpolation(p1x, t_c, t_r1, t_r2, p2x)
                lidar_annotations[ii]['bbox']['position'][1] = self.__linear_interpolation(p1y, t_c, t_r1, t_r2, p2y)
    # __linear_interpolation(self, p1, t_c, t_r1, t_r2, p2)
            except:
                pass
            
    lidar_bev_image = [1152, 1152]

    h_width = lidar_bev_image[0]/2.0
    h_height = lidar_bev_image[1]/2.0
    cell_res_x = 100.0/h_width
    cell_res_y = 100.0/h_height

    # convert meters to pixel
    RadarToLidar[3, 0] = RadarToLidar[3, 0] / cell_res_x
    RadarToLidar[3, 1] = RadarToLidar[3, 1] / cell_res_y
    lidar_annotations = transform_annotations(lidar_annotations, RadarToLidar)
    return lidar_annotations

def __get_projected_bbox(bb, rotation, cameraMatrix, extrinsic, obj_height=2):
    rotation = np.deg2rad(-rotation)
    res = 0.173611
    cx = bb[0] + bb[2] / 2
    cy = bb[1] + bb[3] / 2
    T = np.array([[cx], [cy]])
    pc = 0.2
    bb = [bb[0]+bb[2]*pc, bb[1]+bb[3]*pc, bb[2]-bb[2]*pc, bb[3]-bb[3]*pc]

    R = np.array([[np.cos(rotation), -np.sin(rotation)], [np.sin(rotation), np.cos(rotation)]])

    points = np.array([[bb[0], bb[1]],
                        [bb[0] + bb[2], bb[1]],
                        [bb[0] + bb[2], bb[1] + bb[3]],
                        [bb[0], bb[1] + bb[3]],
                        [bb[0], bb[1]],
                        [bb[0] + bb[2], bb[1] + bb[3]]]).T

    points = points - T
    points = np.matmul(R, points) + T
    points = points.T

    points[:, 0] = points[:, 0] - 576
    points[:, 1] = 576 - points[:, 1]
    points = points * res

    points = np.append(points, np.ones((points.shape[0], 1)) * -1.7, axis=1)
    p1 = points[0, :]
    p2 = points[1, :]
    p3 = points[2, :]
    p4 = points[3, :]

    p5 = np.array([p1[0], p1[1], p1[2] + obj_height])
    p6 = np.array([p2[0], p2[1], p2[2] + obj_height])
    p7 = np.array([p3[0], p3[1], p3[2] + obj_height])
    p8 = np.array([p4[0], p4[1], p4[2] + obj_height])
    points = np.array([p1, p2, p3, p4, p1, p5, p6, p2, p6, p7, p3, p7, p8, p4, p8, p5, p4, p3, p2, p6, p3, p1])

    points = np.matmul(np.append(points, np.ones((points.shape[0], 1)), axis=1), extrinsic.T)

    points = (points / points[:, 3, None])[:, 0:3]

    filtered_indices = []
    for i in range(points.shape[0]):
        if (points[i, 2] > 0 and points[i, 2] < 100):
            filtered_indices.append(i)

    points = points[filtered_indices]

    fx = cameraMatrix[0, 0]
    fy = cameraMatrix[1, 1]
    cx = cameraMatrix[0, 2]
    cy = cameraMatrix[1, 2]

    xIm = np.round((fx * points[:, 0] / points[:, 2]) + cx).astype(np.int)
    yIm = np.round((fy * points[:, 1] / points[:, 2]) + cy).astype(np.int)

    proj_bbox_3d = []
    for ii in range(1, xIm.shape[0]):
        proj_bbox_3d.append([xIm[ii], yIm[ii]])
    proj_bbox_3d = np.array(proj_bbox_3d)
    return proj_bbox_3d


def project_bboxes_to_camera(annotations, intrinsict, extrinsic):
    bboxes_3d = []
    for object in annotations:
        obj = {}
        class_name = object['class_name']
        obj['class_name'] = class_name
        obj['id'] = (object['id'] if 'id' in object.keys() else 0)
        height = heights[class_name]
        bb = object['bbox']['position']
        rotation = object['bbox']['rotation']
        bbox_3d = __get_projected_bbox(bb, rotation, intrinsict, extrinsic, height)
        obj['bbox_3d'] = bbox_3d
        bboxes_3d.append(obj)

    return bboxes_3d


def vis_3d_bbox_cam(image, bboxes_3d, pc_size=0.7):
    vis_im = np.copy(image)
    for obj in bboxes_3d:
        bbox_3d = obj['bbox_3d']
        for ii in range(len(bbox_3d)):
            color = colors[obj['class_name']]
            vis_im = cv2.line(vis_im, (bbox_3d[ii - 1][0], bbox_3d[ii - 1][1]),
                            (bbox_3d[ii][0], bbox_3d[ii][1]), (np.array(color) * 255).astype(np.int).tolist(), 1)

    return vis_im

def get_rectfied(left_im, right_im):
    (leftRectification, rightRectification, leftProjection,
     rightProjection, dispartityToDepthMap, leftROI, rightROI) = cv2.stereoRectify(
        cameraMatrix1=left_cam_mat,
        distCoeffs1=left_cam_dist,
        cameraMatrix2=right_cam_mat,
        distCoeffs2=right_cam_dist,
        imageSize=tuple(left_cam_res),
        R=stereoR,
        T=stereoT,
        flags=cv2.CALIB_ZERO_DISPARITY,
        alpha=0
    )

    leftMapX, leftMapY = cv2.initUndistortRectifyMap(
        left_cam_mat,
        left_cam_dist,
        leftRectification,
        leftProjection, tuple(left_cam_res), cv2.CV_32FC1)

    rightMapX, rightMapY = cv2.initUndistortRectifyMap(
        right_cam_mat,
        left_cam_dist,
        rightRectification,
        rightProjection, tuple(left_cam_res), cv2.CV_32FC1)

    fixedLeft = cv2.remap(left_im, leftMapX,
                              leftMapY, cv2.INTER_LINEAR)
    fixedRight = cv2.remap(right_im, rightMapX,
                               rightMapY, cv2.INTER_LINEAR)

    return fixedLeft, fixedRight, dispartityToDepthMap


def vis_bbox_cam(image, bboxes_3d, pc_size=0.7):
    vis_im = np.copy(image)
    for obj in bboxes_3d:
        color = colors[obj['class_name']]
        bb = np.zeros((4))
        if obj['bbox_3d'].shape[0] > 0:
            bb[0] = np.min(obj['bbox_3d'][:, 0])
            bb[1] = np.min(obj['bbox_3d'][:, 1])
            bb[2] = np.max(obj['bbox_3d'][:, 0])
            bb[3] = np.max(obj['bbox_3d'][:, 1])
            wid = bb[2] - bb[0]
            # hei = bb[3] - bb[1]
            bb[0] += wid*(1.0 - pc_size)
            bb[2] -= wid*(1.0 - pc_size)
            bb = bb.astype(np.int)
            vis_im = cv2.rectangle(vis_im, (bb[0], bb[1]), (bb[2], bb[3]), (255, 0, 0))
            #vis_im = cv2.rectangle(vis_im, (bb[0], bb[1]), (bb[2], bb[3]), (np.array(color) * 255))

    return vis_im


def vis_bbox_cam_annotation(image, bboxes_3d, pc_size=0.7):
    vis_im = np.copy(image)
    bb_list_total = []
    for obj in bboxes_3d:
        color = colors[obj['class_name']]
        bb = np.zeros((4))
        if obj['bbox_3d'].shape[0] > 0:
            bb[0] = np.min(obj['bbox_3d'][:, 0])
            bb[1] = np.min(obj['bbox_3d'][:, 1])
            bb[2] = np.max(obj['bbox_3d'][:, 0])
            bb[3] = np.max(obj['bbox_3d'][:, 1])
            wid = bb[2] - bb[0]
            # hei = bb[3] - bb[1]
            bb[0] += wid*(1.0 - pc_size)
            bb[2] -= wid*(1.0 - pc_size)
            bb = bb.astype(np.int)
            
            bb_list = (bb[0], bb[1], bb[2], bb[3])
            bb_list_total.append(bb_list)
    
    return bb_list_total

def indent(elem, level=0):
    i = "\n" + level*"  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [3]:
# Config


#
radar_calib = {"T" : [0.0, 0.0, 0.0], "R" : [0.0, 0.0, 0.0]}
lidar_calib = {"T" : [0.6003, -0.120102, 0.250012], "R" : [0.0001655, 0.000213, 0.000934]}

RadarT = np.array(radar_calib['T'])
RadarR = np.array(radar_calib['R'])
LidarT = np.array(lidar_calib['T'])
LidarR = np.array(lidar_calib['R'])
        
RadarToLidarR = RadarR - LidarR
RadarToLidarT = RadarT - LidarT

RadarToLidar = transform(RadarToLidarR, RadarToLidarT)

#
heights = {'car': 1.5,'bus': 3, 'truck': 2.5,'pedestrian': 1.8, 'van': 2, 'group_of_pedestrians': 1.8,
        'motorbike': 1.5,'bicycle': 1.5,'vehicle': 1.5}

#
left_cam_calib = {"T" : [0.34001, -0.06988923, 0.287893], "R" : [1.278946, -0.530201, 0.000132],
                 "fx" : 3.379191448899105e+02,
                 "fy" : 3.386957068549526e+02,
                 "cx" : 3.417366010946575e+02,
                 "cy" : 2.007359735313929e+02,
                 "k1" : -0.183879883467351,
                 "k2": 0.0308609205858947,
                 "k3": 0,
                 "p1": 0,
                 "p2": 0,
                 "res": [672, 376]
                 }

fxl = left_cam_calib['fx']
fyl = left_cam_calib['fy']
cxl = left_cam_calib['cx']
cyl = left_cam_calib['cy']
left_cam_mat = np.array([[fxl, 0, cxl], [0, fyl, cyl], [0,  0,  1]])

LeftT = np.array(left_cam_calib['T'])
LeftR = np.array(left_cam_calib['R'])
RadarToLeftT = RadarT - LeftT
RadarToLeftR = RadarR - LeftR
RadarToLeft = transform(RadarToLeftR, RadarToLeftT)

#
colors = {'car': (1, 0, 0),'bus': (0, 1, 0),'truck': (0, 0, 1),'pedestrian': (1.0, 1.0, 0.0), 'van': (1.0, 0.3, 0.0),
        'group_of_pedestrians': (1.0, 1.0, 0.3),'motorbike': (0.0, 1.0, 1.0),'bicycle': (0.3, 1.0, 1.0),
        'vehicle': (1.0, 0.0, 0.0)}

#
k1l = left_cam_calib['k1']
k2l = left_cam_calib['k2']
p1l = left_cam_calib['p1']
p2l = left_cam_calib['p2']

left_cam_dist = np.array([k1l, k2l, p1l, p2l])

stereo_calib = {"TX" : -120.7469, 
                "TY" : 0.1726,
                "TZ" : 1.1592,
                 "CV" : 0.0257154,
                 "RX" : -0.0206928,
                 "RZ" : -0.000595637,
                 "R": [[0.999983541478846,   0.000655753417350, -0.005699715684273],
                       [-0.000622470939159,   0.999982758359834,   0.005839136322126],
                       [0.005703446445424, -0.005835492311203,   0.9999667083098977]]
                 }

stereoR = np.array(stereo_calib['R'])
stereoT = np.array([stereo_calib['TX'], stereo_calib['TY'], stereo_calib['TZ']])

#
right_cam_calib = {"T" : [0.4593822, -0.0600343, 0.287433309324], "R" : [0.8493049332, 0.37113944, 0.000076230],
                 "fx" : 337.873451599077,
                 "fy" : 338.530902554779,
                 "cx" : 329.137695760749,
                 "cy" : 186.166590759716,
                 "k1" : -0.181771143569008,
                 "k2": 0.0295682692890613,
                 "k3": 0,
                 "p1": 0,
                 "p2": 0,
                 "res": [672, 376]
                 }

fxr = right_cam_calib['fx']
fyr = right_cam_calib['fy']
cxr = right_cam_calib['cx']
cyr = right_cam_calib['cy']
p2r = right_cam_calib['p2']
k1r = right_cam_calib['k1']
k2r = right_cam_calib['k2']
p1r = right_cam_calib['p1']

right_cam_mat = np.array([[fxr, 0, cxr], [0, fyr, cyr], [0,  0,  1]])
right_cam_dist = np.array([k1r, k2r, p1r, p2r])

left_cam_res = left_cam_calib['res']

/tmp/ipykernel_31738/1249577507.py:31: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Rx = np.array([[1, 0, 0], [0, np.cos(thetaX), -np.sin(thetaX)], [0, np.sin(thetaX), np.cos(thetaX)]]).astype(np.float)
/tmp/ipykernel_31738/1249577507.py:41: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Rz = np.array([[np.cos(thetaZ), -np.sin(thetaZ), 0]

In [20]:
import natsort

file_root = '/home/aimotion/Desktop/Radiate/data/etc/xml_data_camera/radar_xml'
file_list = os.listdir(file_root)
file_list_sorted = natsort.natsorted(file_list)
#print(file_list_sorted)

for i in range(len(file_list_sorted)):
    filePath = '/home/aimotion/Desktop/Radiate/data/etc/xml_data_camera/radar_xml/'+file_list_sorted[i]
    file = minidom.parse(filePath)
    models = file.getElementsByTagName('file_name')
    for elem in models:
        print(elem.firstChild.data)
        radar_img_path = elem.firstChild.data
    radar_img_path[:radar_img_path.rfind("N")]
    radar_annotation_path = radar_img_path[:radar_img_path.rfind("N")] + 'annotations/annotations.json'
    
    # Parameters
    time_now = int(radar_img_path[radar_img_path.rfind(".")-6:radar_img_path.rfind(".")]) - 1     # radar image frame time
    folder = radar_img_path[:radar_img_path.rfind("N")]
    radar_annotation_path = folder + 'annotations/annotations.json'
    #radar_time_path
    camera_time_path = folder + 'zed_left.txt'

    f = open(radar_annotation_path)
    annotations_json = json.load(f)

    timestamp_path = folder+'Navtech_Cartesian.txt'
    with open(timestamp_path, "r") as file:
        lines = file.readlines()
        timestamps = {'frame': [], 'time': []}
        for line in lines:
            words = line.split()
            timestamps['frame'].append(int(words[1]))
            timestamps['time'].append(__timestamp_format(words[3]))
    timestamps
    timestamp_radar = timestamps

    # Current time
    current_time = timestamp_radar['time'][time_now]
    t = []
 
    for ii in timestamp_radar['frame']:
        t.append(current_time)
    
    # Find a radar time index which is closest to the current time
    a = timestamp_radar['time']
    b = t
    ind = np.argmin(np.abs([ai - bi for ai, bi in zip(a, b)]))
    id_radar = timestamp_radar['frame'][ind]
    ts_radar = timestamp_radar['time'][ind]
    #print(id_radar) # frame number of pic closets to the current time
    #print(ts_radar)

    # t2
    if (len(timestamp_radar['time']) > id_radar + 1):
        t2 = timestamp_radar['time'][id_radar + 1]
    
    # 
    annotations = {}
    annotations['lidar_bev_image'] = get_lidar_annotations(id_radar, False, t, ts_radar, t2)

    #
    bboxes_3d = project_bboxes_to_camera(annotations['lidar_bev_image'],left_cam_mat, RadarToLeft)
    annotations['camera_left_rect'] = bboxes_3d
    output = {}
    output['annotations'] = annotations

    # Load all camera timestamps
    with open(camera_time_path, "r") as file:
        lines_c = file.readlines()
        timestamps_c = {'frame': [], 'time': []}
        for line in lines_c:
            words_c = line.split()
            timestamps_c['frame'].append(int(words_c[1]))
            timestamps_c['time'].append(__timestamp_format(words_c[3]))
    timestamps_c
    timestamp_camera = timestamps_c


    # Find a camera time index which is closest to the current time
    a = timestamp_camera['time']
    b = t
    ind_c = np.argmin(np.abs([ai - bi for ai, bi in zip(a, b)]))
    id_camera = timestamp_camera['frame'][ind_c]
    ts_camera = timestamp_camera['time'][ind_c]
    #print(id_camera) # frame number of pic closets to the current time
    #print(ts_camera)


    sequence_path = folder
    str_format = '{:06d}'

    im_left_path = os.path.join(sequence_path, 'zed_left', str_format.format(id_camera) + '.png')
    im_right_path = os.path.join(sequence_path, 'zed_right', str_format.format(id_camera) + '.png')

    im_left = cv2.imread(im_left_path)
    im_right = cv2.imread(im_right_path)
    im_left_rect, im_right_rect, disp_to_depth = get_rectfied(im_left, im_right)

    output['camera_left_rect'] = im_left_rect

    camera_bboxs = vis_bbox_cam_annotation(output['camera_left_rect'], output['annotations']['camera_left_rect'])
    print(camera_bboxs)
    
    if not camera_bboxs == [] :      
        # Create XML files
        root = Element('annotation')
        head = SubElement(root, "file_name").text = im_left_path
        head = SubElement(root, "file_name_radar").text = file_list_sorted[i]
        head = SubElement(root, "image_id").text = str(i)
        head = SubElement(root, "size")
        SubElement(head, "width").text = "672"
        SubElement(head, "height").text = "376"         
        number_of_objects = len(camera_bboxs)
        
        for n in range (number_of_objects):
            if camera_bboxs[n][0] >= 0 and camera_bboxs[n][0] <= 672 and camera_bboxs[n][1] >= 0 and camera_bboxs[n][1] <= 376 and camera_bboxs[n][2] >= 0 and camera_bboxs[n][2] <= 672 and camera_bboxs[n][3] >= 0 and camera_bboxs[n][3] <= 376 : 
                head = SubElement(root, "object")
                head1 = SubElement(head, "name").text = "vehicle"
                head2 = SubElement(head, "bndbox")
                SubElement(head2, "xmin").text = str(camera_bboxs[n][0])
                SubElement(head2, "ymin").text = str(camera_bboxs[n][1])
                SubElement(head2, "xmax").text = str(camera_bboxs[n][2])
                SubElement(head2, "ymax").text = str(camera_bboxs[n][3])
        
        if camera_bboxs[n][0] >= 0 and camera_bboxs[n][0] <= 672 and camera_bboxs[n][1] >= 0 and camera_bboxs[n][1] <= 376 and camera_bboxs[n][2] >= 0 and camera_bboxs[n][2] <= 672 and camera_bboxs[n][3] >= 0 and camera_bboxs[n][3] <= 376 : 
            indent(root)   
            dump(root)     
            tree = ElementTree(root)
            tree.write('/home/aimotion/Desktop/Radiate/data/etc/xml_data_camera/camera_xml/'+'c_'+file_list_sorted[i])


            
            
#
file_root = '/home/aimotion/Desktop/Radiate/data/etc/xml_data_camera/camera_xml/'
file_list = os.listdir(file_root)
file_list_sorted = natsort.natsorted(file_list)
print(file_list_sorted)

for i in range(len(os.listdir(file_root))):
    xml_file = file_root+file_list_sorted[i]
    tree = ET.parse(xml_file)
    root = tree.getroot()
    name_tag = root.findtext("file_name")
    origin = str(name_tag)
    shutil.copy(origin, file_root)    
    
    dirname, basename = os.path.split(origin)
    origin_name = basename
    os.rename('/home/aimotion/Desktop/Radiate/data/etc/xml_data_camera/camera_xml/'+str(origin_name), file_root+file_list_sorted[i][:-4]+'.png')   

/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000016.png
[(314, 194, 321, 201)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/zed_left/000042.png</file_name>
  <file_name_radar>sunny_6.xml</file_name_radar>
  <image_id>0</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>314</xmin>
      <ymin>194</ymin>
      <xmax>321</xmax>
      <ymax>201</ymax>
    </bndbox>
  </object>
</annotation>
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000026.png
[(300, 195, 312, 206)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/zed_left/000080.png</file_name>
  <file_name_radar>sunny_16.xml</file_name_radar>
  <image_id>1</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle</name>
    

/tmp/ipykernel_31738/1249577507.py:159: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  xIm = np.round((fx * points[:, 0] / points[:, 2]) + cx).astype(np.int)
/tmp/ipykernel_31738/1249577507.py:160: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additiona

[(300, 192, 306, 201)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/zed_left/000714.png</file_name>
  <file_name_radar>sunny_111.xml</file_name_radar>
  <image_id>8</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>300</xmin>
      <ymin>192</ymin>
      <xmax>306</xmax>
      <ymax>201</ymax>
    </bndbox>
  </object>
</annotation>
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000244.png
[(299, 192, 304, 200)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/zed_left/000714.png</file_name>
  <file_name_radar>sunny_114.xml</file_name_radar>
  <image_id>9</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>299</xmin>
      <ymin>192</ymin>
      <xmax>304</xmax>
      <ymax>200</

[(404, 195, 423, 206), (310, 190, 318, 200)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/zed_left/000714.png</file_name>
  <file_name_radar>sunny_305.xml</file_name_radar>
  <image_id>25</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>404</xmin>
      <ymin>195</ymin>
      <xmax>423</xmax>
      <ymax>206</ymax>
    </bndbox>
  </object>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>310</xmin>
      <ymin>190</ymin>
      <xmax>318</xmax>
      <ymax>200</ymax>
    </bndbox>
  </object>
</annotation>
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/Navtech_Cartesian/000493.png
[(262, 182, 292, 220), (310, 194, 315, 200)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_0/zed_left/000714.png</file_name>
  <file_name_radar>sunny_326.xml</file_name_radar>
  <image_id>26<

[]
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_1/Navtech_Cartesian/000268.png
[]
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_1/Navtech_Cartesian/000273.png
[]
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_1/Navtech_Cartesian/000324.png
[(294, 191, 310, 209)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_1/zed_left/000721.png</file_name>
  <file_name_radar>sunny_553.xml</file_name_radar>
  <image_id>48</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>294</xmin>
      <ymin>191</ymin>
      <xmax>310</xmax>
      <ymax>209</ymax>
    </bndbox>
  </object>
</annotation>
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_1/Navtech_Cartesian/000332.png
[(253, 188, 285, 231)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_1/zed_le

...

[(272, 197, 294, 228)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_4/zed_left/000722.png</file_name>
  <file_name_radar>sunny_1925.xml</file_name_radar>
  <image_id>188</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>272</xmin>
      <ymin>197</ymin>
      <xmax>294</xmax>
      <ymax>228</ymax>
    </bndbox>
  </object>
</annotation>
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_4/Navtech_Cartesian/000689.png
[(206, 197, 238, 230)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_4/zed_left/000722.png</file_name>
  <file_name_radar>sunny_1936.xml</file_name_radar>
  <image_id>189</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>206</xmin>
      <ymin>197</ymin>
      <xmax>238</xmax>
      <ymax

[(263, 197, 285, 226)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_5/zed_left/000722.png</file_name>
  <file_name_radar>sunny_2182.xml</file_name_radar>
  <image_id>215</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>263</xmin>
      <ymin>197</ymin>
      <xmax>285</xmax>
      <ymax>226</ymax>
    </bndbox>
  </object>
</annotation>
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_5/Navtech_Cartesian/000356.png
[(-7, 206, 129, 379)]
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_5/Navtech_Cartesian/000357.png
[(-76, 220, 11, 376)]
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_5/Navtech_Cartesian/000362.png
[]
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_5/Navtech_Cartesian/000369.png
[(358, 195, 377, 207)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/data

[(293, 195, 304, 207)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_5/zed_left/000722.png</file_name>
  <file_name_radar>sunny_2368.xml</file_name_radar>
  <image_id>234</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>293</xmin>
      <ymin>195</ymin>
      <xmax>304</xmax>
      <ymax>207</ymax>
    </bndbox>
  </object>
</annotation>
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_5/Navtech_Cartesian/000605.png
[(299, 194, 307, 202)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_5/zed_left/000722.png</file_name>
  <file_name_radar>sunny_2374.xml</file_name_radar>
  <image_id>235</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>299</xmin>
      <ymin>194</ymin>
      <xmax>307</xmax>
      <ymax

[(310, 196, 323, 211)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_6/zed_left/000722.png</file_name>
  <file_name_radar>sunny_2565.xml</file_name_radar>
  <image_id>257</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>310</xmin>
      <ymin>196</ymin>
      <xmax>323</xmax>
      <ymax>211</ymax>
    </bndbox>
  </object>
</annotation>
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_6/Navtech_Cartesian/000452.png
[]
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_6/Navtech_Cartesian/000474.png
[(249, 196, 269, 219)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_6/zed_left/000722.png</file_name>
  <file_name_radar>sunny_2598.xml</file_name_radar>
  <image_id>259</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle

[]
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000453.png
[(314, 194, 320, 200)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/zed_left/000719.png</file_name>
  <file_name_radar>sunny_2836.xml</file_name_radar>
  <image_id>279</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle</name>
    <bndbox>
      <xmin>314</xmin>
      <ymin>194</ymin>
      <xmax>320</xmax>
      <ymax>200</ymax>
    </bndbox>
  </object>
</annotation>
/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/Navtech_Cartesian/000463.png
[(293, 195, 305, 206)]
<annotation>
  <file_name>/home/aimotion/Desktop/Radiate/dataset/dataset_original/junction_1_7/zed_left/000719.png</file_name>
  <file_name_radar>sunny_2846.xml</file_name_radar>
  <image_id>280</image_id>
  <size>
    <width>672</width>
    <height>376</height>
  </size>
  <object>
    <name>vehicle